## Lesson 9 - SQL and BigQuery

So you have just learnt about how to use python to carry out some basic functionality. There is a lot more to learn about when it comes to writing code, especially the analysis of OpenPrescribing data. But before we do that, we thought you might be itching to actually get some of this knowledge rich data straight away. And why not. Hopefully by now you have your BigQuery credentials from one of the admin team members. If you do, then please follow the step-by-step guide in the [README.md](../README.md) file of this repo. If you don't have your credentials, please give the admin team a gentle nudge. 

Once set up, try running the code below.

In [ ]:
from ebmdatalab import bq
from pathlib import Path

DATA_FOLDER = Path("data")

sql = """
    SELECT code, name
    FROM ebmdatalab.hscic.ccgs
    WHERE name IS NOT NULL
    GROUP BY code, name
    """

ccg_names = bq.cached_read(sql, DATA_FOLDER / "ccg_names.csv", use_cache=False)
ccg_names

Now what happened here!

On a high level plain, the above code got some data from OpenPrescribing's BigQuery data platform, and presented to you the data in table format. There is a lot to unpick here, so let's do just that!



## Introduction to databases and Google BigQuery

Most people have used a `spreadsheet`, such as Excel or Google Sheets. A spreadsheet has rows and columns. Each `row` is a single record - perhaps one prescription, one patient visit, or one payment. Each `column` is a type of information - the patient name, the date, the medicine, or the amount. In this sense, a spreadsheet is already behaving like a simple database.

Below is a simple collection of prescription data:

| Patient | Date       | Medicine     | Quantity | Practice  |
|---------|------------|--------------|----------|-----------|
| Alice   | 05/01/2024 | Omeprazole   | 28       | Riverside |
| Ben     | 06/01/2024 | Paracetamol  | 16       | Hilltop   |
| Carla   | 07/01/2024 | Amoxicillin  | 21       | Riverside |
| Dan     | 08/01/2024 | Omeprazole   | 56       | Riverside |
| John    | 09/01/2024 | Omeprazole   | 0        | Hilltop   |
| Helen   | 10/01/2024 | Amoxicillin  | 8        | NULL      |

An issue that we come across however is that spreadsheets only go so far. They work okay for relatively small numbers of rows, around the hundred to thousands mark, but when you reach millions of rows, a spreadsheet can slow down, become difficult to share, and sometimes simply cannot open. This is where `databases` come in. A database is built to handle large amounts of information, keep it structured, and let you ask questions about it.

`Google BigQuery` is one of these databases, but it is designed for `truly massive amounts of data`. Instead of storing the information on your own laptop, it lives in the cloud on Google’s servers.  You do not scroll through it in the way you would with a spreadsheet. Instead, you ask it questions using the `SQL language`, which stands for `Structured Query Language`. 

> By the way, a `cloud` or `server` is basically a computer connected to the internet with massive amounts of storage and computing power.

So, imagine you have a spreadsheet of prescriptions that has grown so large that it will no longer open. With BigQuery, you can still work with it. You could ask, “How many prescriptions were written in 2024?” and BigQuery would return the number in seconds. You could go further and ask, “Break that total down by GP practice” and BigQuery would return a neat table with one row per practice and the prescription totals beside it.

BigQuery is, in short, a database that looks and feels like a supercharged spreadsheet in the cloud. It can handle billions of rows, answer questions in seconds, and allow many people to work with the same information all at the same time.

## Structured Query Language (SQL)

We have already seen that a spreadsheet is like a database. The terminology translation from spreadsheets to databases are: rows are `records` (one prescription, one patient, one order) and columns are `fields` (the patient name, the medicine, the date, the quantity). With a spreadsheet you scroll, filter, and add formulas. With a database you do something similar, but instead of clicking around you use the query language `SQL`.

SQL is how you ask questions about the information inside a database. Instead of saying “filter this column” or “sum that column” with your mouse, you write a short text command. The database then reads the command, searches through the records, and gives you the result.


### Give me everything!

Let's say, to keep things very simple, we put the data that is in the spreadsheet above into a database, either on your laptop or on Google BigQuery. We put this data into an area in the database called a table. Think of a table as the individual spreadsheet tabs on your home computer spreadsheet. We decide on a name for this table of "prescriptions" (it could be anything you like really). Now let's say you want to get hold of all of the data in the "prescriptions" table. What you would say, in the SQL (pronounced "sequel") language is "give me everything you have from the prescriptions table". The exact `syntax` for this message to the database in the SQL language looks like this:

```sql
SELECT *
FROM prescriptions;
```

The asterisk `*` means `everything`. 

The word `syntax` just means the `wording` that we use for the SQL language.

The results of the above SQL question would look like this:

| Patient | Date       | Medicine     | Quantity | Practice  |
|---------|------------|--------------|----------|-----------|
| Alice   | 05/01/2024 | Omeprazole   | 28       | Riverside |
| Ben     | 06/01/2024 | Paracetamol  | 16       | Hilltop   |
| Carla   | 07/01/2024 | Amoxicillin  | 21       | Riverside |
| Dan     | 08/01/2024 | Omeprazole   | 56       | Riverside |
| John    | 09/01/2024 | Omeprazole   | 0        | Hilltop   |
| Helen   | 10/01/2024 | Amoxicillin  | 8        | NULL      |

### No names please

Now how about if you did not want to know about patient names, or dates, but only how many drugs are being prescribed. This would be really useful if you want to remove personal details about who is actually prescribed what. If that was the case, then we could ask to only get the `Medicine` and `Quantity` columns. So the SQL for this question looks like this:

```sql
SELECT Medicine, Quantity
FROM prescriptions;
```

Translating the above SQL into human reads as "give me the medicine and quantity fields from the prescriptions database".

By the way, when we ask the database a question using the SQL language, we say we `query` the database.

The above SQL query would return the below result:

| Medicine     | Quantity |
|--------------|----------|
| Omeprazole   | 28       |
| Paracetamol  | 16       |
| Amoxicillin  | 21       |
| Omeprazole   | 56       |
| Omeprazole   | 0        |
| Amoxicillin  | 8        |

### Only a single type of drug please

Perhaps you only want the rows with Omeprazole in them. We would ask in SQL of the database "look at all of the data from the prescription table and return only the rows that have Omeprazole in the Medicine column":

```sql
SELECT *
FROM prescriptions
WHERE Medicine = 'Omeprazole'
```

This SQL query would give you the below result:

| Patient | Date       | Medicine     | Quantity | Practice  |
|---------|------------|--------------|----------|-----------|
| Alice   | 05/01/2024 | Omeprazole   | 28       | Riverside |
| Dan     | 08/01/2024 | Omeprazole   | 56       | Riverside |
| John    | 09/01/2024 | Omeprazole   | 0        | Hilltop   |

### Count them all up!

Perhaps you want to know how many drugs you have prescribed from the data you have in front of you. You can use the `SUM` function for that!

```sql
SELECT SUM(Quantity)
FROM prescriptions;
```

Translation: "get me the sum of the quantity field from the prescriptions table".

You would get a result of:


```bash
129
```

### One group please!

How about if you want to sum up all Medicines of each type, eg all omeprazole, and all paracetamol, etc? Well, to do this you need to ask the database to `group` the data by the Medicine's column, and then return the results. You can do this as below:

```sql
SELECT Medicine, SUM(Quantity)
FROM prescriptions
GROUP BY Medicine
```

Translation: "give me the Medicine field and the SUM of the Quantity field from the prescriptions table and group the results by the values in the Medicine field".

And your SQL query returns:

| Medicine     | SUM(Quantity) |
|--------------|---------------|
| Omeprazole   | 84            |
| Paracetamol  | 16            |
| Amoxicillin  | 29            |

### How about two groups?

And how about if you want to summarise in groups and subgroups, for example 1st by practice and then by medicine name. The SQL for this would look like this:

```sql
SELECT Practice, Medicine, SUM(Quantity)
FROM prescriptions
GROUP BY Practice, Medicine;
```

Translation: "give me the Practice and Medicine fields and also the SUM of the Quantity field. Get this data from the prescriptions table and then group all of the results 1st by Practice and then subgroup by Medicine".

Resulting in this data:

| Practice    | Medicine    | SUM(Quantity) |
|-------------|-------------|---------------|
| Riverside   | Omeprazole  | 84            |
| Riverside   | Amoxicillin | 21            |
| Hilltop     | Paracetamol | 16            |
| Hilltop     | Omeprazole  | 0             |
| NULL        | Amoxicillin | 8             |

### Let's neaten that heading

You may not like the headings from the last output, especially that unsightly `SUM(Quantity)` one! Well you can change any section name as you like. To change the name of a field heading, we use the `AS` keyword.

NB: This only changes the printed out heading, not the heading in the data itself.

```sql
SELECT Practice AS "GP surgery", Medicine AS "Drug name", SUM(Quantity) AS "Total"
FROM prescriptions
GROUP BY Practice, Medicine;
```

Translation: We will let you work that one out!

| GP surgery  | Drug name   | Total         |
|-------------|-------------|---------------|
| Riverside   | Omeprazole  | 84            |
| Riverside   | Amoxicillin | 21            |
| Hilltop     | Paracetamol | 16            |
| Hilltop     | Omeprazole  | 0             |
| NULL        | Amoxicillin | 8             |

### What's `NULL` got to do with it!

You most likely do not care about a surgery with no name (eg `NULL`). Let's remove this location from the above results:

```sql
SELECT Practice AS "GP surgery", Medicine AS "Drug name", SUM(Quantity) AS "Total"
FROM prescriptions
WHERE Practice IS NOT NULL
GROUP BY Practice, Medicine;
```

NB: We use `WHERE` to evaluate something. `IS NOT` does what it says on the tin! `NULL` is a computer term for `value not present`.

| GP surgery  | Drug name   | Total         |
|-------------|-------------|---------------|
| Riverside   | Omeprazole  | 84            |
| Riverside   | Amoxicillin | 21            |
| Hilltop     | Paracetamol | 16            |
| Hilltop     | Omeprazole  | 0             |


### Put it all together!

So, look again at the SQL query that we sent to OpenPrescribing's BigQuery database.

```sql
SELECT code, name
FROM ebmdatalab.hscic.ccgs
WHERE name IS NOT NULL
GROUP BY code, name
```

Can you work out what it is trying to say?

If you have been following along with the above examples, you should be able to translate it into something like this:

```text
Get the "code" and "name" fields from the "ebmdatalab.hscic.ccgs" table. Do not give me any rows where the name column is empty (eg NULL). Group the results by the code and then name columns
```

### What are these dots doing?

One final thing for this lesson. You may have noticed the full stops `.` in the FROM line above (ie `FROM ebmdatalab.hscic.ccgs`). These full spots are used by BigQuery (and other databases) as a way to reference tables and the way they are stored. The first element `ebmdatalab` is the `project` your table is stored in, `hscic` is the dataset (basically a folder of tables), and `ccgs` is the actual table you are interested in.

`CCG` by the way stands for `Clinical Commisioning Group`. These small regions (211 to be exact) were set up by NHS England to manage local services. The data in the ccgs table is now slightly mixed around since CCG were disbanded.

### Moving on

OK, that is SQL and BigQuery covered. But what about all of that other `code` in the BigQuery example above. Well, we will cover that in lesson 10!